# Pymaceuticals Inc.
---

### Analysis

- Overall, Capomulin is more effective in reducing tumour volume than most of the other drug regimens.
  - The only drug that gives the same effectiveness is Ramicane.
  - The variance and spread of the data for Capomulin is also lower than the other drugs, making the accuracy of the drug higher than the rest.
- There is a strong positive correlation between the average tumour volume and weight, with the correlation coefficient being 0.84.

## Prepare the Data

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
complete_df = pd.merge(mouse_metadata, study_results, on='Mouse ID', how='left')

# Display the data table for preview
complete_df.head()

In [ ]:
# Checking the number of mice.
mice_count = complete_df['Mouse ID'].nunique()
mice_count

In [ ]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicates = complete_df.duplicated(subset=['Mouse ID','Timepoint'], keep=False)
duplicates = complete_df[duplicates]
duplicates = duplicates['Mouse ID'].unique()
duplicates

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
same_mice = complete_df.loc[complete_df['Mouse ID']==duplicates[0],:]
same_mice

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_df = complete_df.loc[complete_df['Mouse ID']!=duplicates[0],:]
clean_df

In [ ]:
# Checking the number of mice in the clean DataFrame.
mice_count2 = clean_df['Mouse ID'].nunique()
mice_count2

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
drug_group = clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)']
tumour_mean = drug_group.mean()
tumour_median = drug_group.median()
tumour_variance = drug_group.var()
tumour_std = drug_group.std()
tumour_sem = drug_group.sem()

# Assemble the resulting series into a single summary DataFrame.
tumour_summary = pd.DataFrame({'Mean Tumour Volume': tumour_mean,
                               'Median Tumour Volume': tumour_median,
                               'Tumour Volume Variance': tumour_variance,
                               'Tumour Volume Std. Dev.': tumour_std,
                               'Tumour Volume Std. Err.': tumour_sem})
tumour_summary

In [ ]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line
tumour_stats = clean_df.groupby('Drug Regimen').agg({'Tumor Volume (mm3)':['mean', 'median', 'var', 'std', 'sem']})
tumour_stats

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.
total_rows = clean_df.groupby('Drug Regimen')['Timepoint'].count().sort_values(ascending=False)
drug_bar = total_rows.plot(kind='bar', title='Number of Observed Timepoints per Drug Regimen', 
                           xlabel='Drug Regimen', ylabel='# of Observed Mouse Timepoints')
plt.show()

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.

plt.bar(total_rows.index, total_rows)
plt.xticks(rotation=90)
plt.title("Number of Observed Timepoints per Drug Regimen")
plt.xlabel("Drug Regimen")
plt.ylabel("# of Observed Mouse Timepoints")

plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
genders = clean_df.groupby('Sex')['Mouse ID'].count()
genders.plot(kind='pie', title='Gender Split',ylabel='Sex', autopct='%.1f%%', startangle=180)
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
plt.pie(genders, autopct='%.1f%%', labels=['Female', 'Male'], startangle=180)
plt.title('Gender Split')
plt.ylabel('Sex')

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse
max_group = pd.DataFrame(clean_df.groupby('Mouse ID')['Timepoint'].max())

# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint
merged_max = pd.merge(max_group, clean_df, on=['Mouse ID','Timepoint'])
merged_max


In [ ]:
# Filter to only included the necessary drug regimens
treatments = ['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin']
merged_max_filter = merged_max[merged_max['Drug Regimen'].isin(treatments)]
merged_max_filter

In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.
boxplot = merged_max_filter.boxplot(column=['Tumor Volume (mm3)'], by='Drug Regimen', grid=False)
plt.suptitle('Final Tumor Volume of the Most Promising Drug Regimens')
plt.title(None)
plt.ylabel('Final Tumor Volume (mm3)')
plt.show()

## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin
capomulin = clean_df[clean_df['Drug Regimen']=='Capomulin']
capomulin_mouse = capomulin['Mouse ID'].iloc[0]
capomulin_filter = capomulin[capomulin['Mouse ID']==capomulin_mouse]
line = capomulin_filter.plot.line('Timepoint', 'Tumor Volume (mm3)')
plt.title('Capomulin Treatment of Mouse S185')
plt.xlabel('Timepoint (days)')
plt.ylabel('Tumor Volume (mm3)')
line.get_legend().remove()
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen
capomulin_id = capomulin.groupby('Mouse ID')
average = capomulin_id[['Tumor Volume (mm3)', 'Weight (g)']].mean()
scatter = average.plot.scatter('Weight (g)', 'Tumor Volume (mm3)')
plt.title('Average Tumor Volume Against Weight')
plt.ylabel('Average Tumor Volume (mm3)')

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen
res = st.linregress(average['Weight (g)'], average['Tumor Volume (mm3)'])

# Plot the linear regression on the scatter plot
scatter = average.plot.scatter('Weight (g)', 'Tumor Volume (mm3)')
plt.plot(average['Weight (g)'], res.intercept + res.slope*average['Weight (g)'], 'r')
plt.title('Average Tumor Volume Against Weight')
plt.ylabel('Average Tumor Volume (mm3)')
print(f"The correlation between mouse weight and the average tumor volume is {round(res.rvalue,2)}.")